In [1]:
!pwd

/data/work/lvjinwei/work/nlp/fasttext/yinniText/text_model/method5


In [1]:
import numpy as np
import logging
import gensim
import datetime
import pandas as pd
import fastText
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from project_demo.tools.dbUtil import *
from project_demo.tools.multi_apply import *
from project_demo.tools.timeParse import *
from project_demo.tools.evaluate import *
import csv
import sys
sys.path.append('../../')
from Utils.utils import *
from Utils.TextClass import *

def is_zeros(array):
    flag = True
    for i in array:
        if i != 0:
            flag = False
            break
    return flag

def get_vector_mean(content):
    word_list = content.split(' ')
    vector_list = [model.get_word_vector(w) for w in word_list]
    vector_list_new = np.array(list(filter(lambda x: not is_zeros(x),vector_list)))
    vector = np.array(vector_list_new).sum(axis=0)/np.array(vector_list_new).shape[0]
    if type(vector) == np.float64:
        vector = [np.nan]*100
    return vector
    

def vector_feature(dataset,get_vector):
    vector_list = list(map(lambda x: get_vector(x), dataset['content']))
    datavec = pd.DataFrame(vector_list)
    return datavec

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv('./train_seg.tsv',sep='\t')
test = pd.read_csv('./test_seg.tsv',sep='\t')
vaild = pd.read_csv('./valid_seg.tsv',sep='\t')

In [5]:
model = fastText.train_unsupervised("train_fasttext.tsv", epoch=10, dim=100, lr=0.08, 
                                    wordNgrams=2, verbose=2, minCount=2
                                    pretrainedVectors="/data/work/wangkai1/yinnibr/text_model/pretrain/wiki.id.vec")

In [31]:
train_datavec=vector_feature(train,get_vector_mean)
test_datavec=vector_feature(test,get_vector_mean)
vaild_datavec=vector_feature(vaild,get_vector_mean)
train_datavec['target'] = train['target']
test_datavec['target'] = test['target']
vaild_datavec['target'] = vaild['target']

lgbm:

In [32]:
from sklearn import metrics
from scipy.stats import ks_2samp
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import LabelBinarizer
from sklearn2pmml.decoration import CategoricalDomain, ContinuousDomain
from sklearn2pmml import PMMLPipeline
from project_demo.tools.evaluate import *
from project_demo.tools.optimize import *
from bayes_opt import BayesianOptimization
import time
import pandas as pd
import numpy as np
from sklearn.externals import joblib
import pickle
%matplotlib inline

DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [33]:
def modelWithCv(model, x_array, y_array, cv=5):
    model.fit(x_array, y_array)
    
    dtrain_predictions = model.predict(x_array)
    dtrain_predprob = model.predict_proba(x_array)[:,1]
    
    print("--AUC Score (Train): %f" % metrics.roc_auc_score(y_array, dtrain_predprob))
    print ("class metrics:")
    print (metrics.classification_report(y_array, dtrain_predictions))
    
    cv_score = cross_val_score(model,x_array, y_array, cv=cv, scoring = 'roc_auc')
    print("--CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" %(np.mean(cv_score), np.std(cv_score), np.min(cv_score), np.max(cv_score)))

def auc_ks(model, dataframemapper, trainset, testset, varlist,train_only=False):
    if train_only == False:
        df_test=dataframemapper.transform(testset)
        predprob = pd.DataFrame(model.predict_proba(df_test[varlist])[:,1], columns = ['predprob'])
        predprob['ytrue'] = testset['target'].values
        auc = metrics.roc_auc_score(y_score=predprob['predprob'], y_true=predprob['ytrue'])
        print('AUC On Test is: {}'.format(auc))
        print('KS On Test is: {}'.format(cal_ks_scipy(predprob['predprob'], predprob['ytrue'])))
    
    df_train=dataframemapper.transform(trainset)
    predprob = pd.DataFrame(model.predict_proba(df_train[varlist])[:,1], columns = ['predprob'])
    predprob['ytrue'] = trainset['target'].values
    auc = metrics.roc_auc_score(y_score=predprob['predprob'], y_true=predprob['ytrue'])
    print('AUC On Train is: {}'.format(auc))
    print('KS On Train is: {}'.format(cal_ks_scipy(predprob['predprob'], predprob['ytrue'])))
    
def continuous_categorical(dataset):
    '''diff continuous and categorical
        
    '''
    continuousDomain = [] # float int
    categoricalDomain = [] # object bool category

    for item in dataset.columns:
        if (dataset[item].dtypes == object)| (dataset[item].dtypes == bool):
            categoricalDomain.append(item)
            dataset[item] = dataset[item].astype(str)
        elif item!='target':
            continuousDomain.append(item)
            dataset[item] = dataset[item]
    print (categoricalDomain)
    return continuousDomain,categoricalDomain

In [34]:
train_datavec.replace(np.nan,-1 ,inplace=True)

continuousDomain, categoricalDomain = continuous_categorical(train_datavec)

test_datavec.replace(np.nan,-1 ,inplace=True)

vaild_datavec.replace(np.nan,-1 ,inplace=True)

dfm = DataFrameMapper([(c, [CategoricalDomain(invalid_value_treatment = 'as_missing',
                                              missing_value_treatment = 'as_value',
                                              missing_value_replacement = 'N/A'), LabelBinarizer()])
                       for c in categoricalDomain] 
                      + 
                     [(c, [ContinuousDomain(invalid_value_treatment = 'as_missing',
                                                     missing_value_treatment = 'as_value',
                                                     missing_value_replacement = -1)])
                      for c in continuousDomain]
                      ,df_out = True)

dfm.fit(train_datavec)

X_train_datavec = dfm.transform(train_datavec)
y_train_datavec = train_datavec['target']

[]


In [35]:
model = LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.8,
        drop_rate=0.1, is_unbalance=True,
        learning_rate=0.01, max_bin=10, max_depth=4,
        max_drop=5, min_child_samples=300, min_child_weight=50,
        min_split_gain=3.5, n_estimators=800, nthread=-1,
        num_leaves=8, objective='binary', reg_alpha=1, reg_lambda=200,
        scale_pos_weight=1, seed=27, sigmoid=1.0, silent=True,
        skip_drop=0.5, subsample=0.7, subsample_for_bin=1000,
        subsample_freq=2, uniform_drop=False, xgboost_dart_mode=False)

In [36]:
%%time
modelWithCv(model, X_train_datavec, y_train_datavec, 2)

varlist=X_train_datavec.columns

auc_ks(model, dfm, train_datavec, test_datavec, varlist)

auc_ks(model, dfm, train_datavec, vaild_datavec, varlist)

/opt/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


--AUC Score (Train): 0.681562
class metrics:
             precision    recall  f1-score   support

          0       0.36      0.66      0.47      5796
          1       0.84      0.60      0.70     17223

avg / total       0.72      0.62      0.64     23019

--CV Score : Mean - 0.6382155 | Std - 0.004562738 | Min - 0.6336527 | Max - 0.6427782
AUC On Test is: 0.6380702351166486
KS On Test is: 0.21621183013588074
AUC On Train is: 0.6815617313135168
KS On Train is: 0.2669845164676394
AUC On Test is: 0.6372969537058191
KS On Test is: 0.20111636119207066
AUC On Train is: 0.6815617313135168
KS On Train is: 0.2669845164676394
CPU times: user 8min 52s, sys: 31.5 s, total: 9min 23s
Wall time: 13.7 s
